# Association analysis sample (Online Retail II)

- Online Retail II Data Set https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

In [1]:
import pandas as pd
import dask.dataframe as dd
import dask

In [2]:
# Dataset
_read_xlsx_delayed = dask.delayed(pd.read_excel)(
    'online_retail_II.xlsx'
)
DF_RAW = dd.from_delayed(_read_xlsx_delayed).compute()

In [3]:
display(DF_RAW.shape[0])
display(DF_RAW.head())

525461

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
display(DF_RAW.isnull().sum())

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [5]:
print(
    DF_RAW['Country'].value_counts().head(15)
)

United Kingdom     485852
EIRE                 9670
Germany              8129
France               5772
Netherlands          2769
Spain                1278
Switzerland          1187
Portugal             1101
Belgium              1054
Channel Islands       906
Sweden                902
Italy                 731
Australia             654
Cyprus                554
Austria               537
Name: Country, dtype: int64
